# **Homework 2: Phoneme Classification**


Objectives:
* Solve a classification problem with deep neural networks (DNNs).
* Understand recursive neural networks (RNNs).

If you have any questions, please contact the TAs via TA hours, NTU COOL, or email to mlta-2023-spring@googlegroups.com

# Download Data
Download data from google drive, then unzip it.

You should have
- `libriphone/train_split.txt`: training metadata
- `libriphone/train_labels`: training labels
- `libriphone/test_split.txt`: testing metadata
- `libriphone/feat/train/*.pt`: training feature
- `libriphone/feat/test/*.pt`:  testing feature

after running the following block.

> **Notes: if the google drive link is dead, you can download the data directly from [Kaggle](https://www.kaggle.com/c/ml2023spring-hw2/data) and upload it to the workspace.**


In [27]:
# !pip install --upgrade gdown

# # Main link
# !gdown --id '1N1eVIDe9hKM5uiNRGmifBlwSDGiVXPJe' --output libriphone.zip
# # !gdown --id '1qzCRnywKh30mTbWUEjXuNT2isOCAPdO1' --output libriphone.zip

# !unzip -q libriphone.zip
# !ls libriphone

# Some Utility Functions
**Fixes random number generator seeds for reproducibility.**

In [28]:
import numpy as np
import torch
import random

def same_seeds(seed):
    random.seed(seed) 
    np.random.seed(seed)  
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed) 
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

**Helper functions to pre-process the training data from raw MFCC features of each utterance.**

A phoneme may span several frames and is dependent to past and future frames. \
Hence we concatenate neighboring phonemes for training to achieve higher accuracy. The **concat_feat** function concatenates past and future k frames (total 2k+1 = n frames), and we predict the center frame.

Feel free to modify the data preprocess functions, but **do not drop any frame** (if you modify the functions, remember to check that the number of frames are the same as mentioned in the slides)

In [29]:
import os
import torch
from tqdm import tqdm

def load_feat(path):
    feat = torch.load(path)
    return feat

def shift(x, n):
    if n < 0:
        left = x[0].repeat(-n, 1)
        right = x[:n]
    elif n > 0:
        right = x[-1].repeat(n, 1)
        left = x[n:]
    else:
        return x

    return torch.cat((left, right), dim=0)

def concat_feat(x, concat_n):
    assert concat_n % 2 == 1 # n must be odd
    if concat_n < 2:
        return x
    seq_len, feature_dim = x.size(0), x.size(1)
    x = x.repeat(1, concat_n) 
    x = x.view(seq_len, concat_n, feature_dim).permute(1, 0, 2) # concat_n, seq_len, feature_dim
    mid = (concat_n // 2)
    for r_idx in range(1, mid+1):
        x[mid + r_idx, :] = shift(x[mid + r_idx], r_idx)
        x[mid - r_idx, :] = shift(x[mid - r_idx], -r_idx)

    return x.permute(1, 0, 2)#.view(seq_len, concat_n * feature_dim)

def preprocess_data(split, feat_dir, phone_path, concat_nframes, train_ratio=0.8, random_seed=1213):
    class_num = 41 # NOTE: pre-computed, should not need change

    if split == 'train' or split == 'val':
        mode = 'train'
    elif split == 'test':
        mode = 'test'
    else:
        raise ValueError('Invalid \'split\' argument for dataset: PhoneDataset!')

    label_dict = {}
    if mode == 'train':
        for line in open(os.path.join(phone_path, f'{mode}_labels.txt')).readlines():
            line = line.strip('\n').split(' ')
            label_dict[line[0]] = [int(p) for p in line[1:]]
        
        # split training and validation data
        usage_list = open(os.path.join(phone_path, 'train_split.txt')).readlines()
        random.seed(random_seed)
        random.shuffle(usage_list)
        train_len = int(len(usage_list) * train_ratio)
        usage_list = usage_list[:train_len] if split == 'train' else usage_list[train_len:]

    elif mode == 'test':
        usage_list = open(os.path.join(phone_path, 'test_split.txt')).readlines()

    usage_list = [line.strip('\n') for line in usage_list]
    print('[Dataset] - # phone classes: ' + str(class_num) + ', number of utterances for ' + split + ': ' + str(len(usage_list)))

    max_len = 3000000
    usage_len = usage_list.__len__()
    # X = torch.empty(max_len, 39 * concat_nframes)
    # X = torch.empty(max_len, concat_nframes, 39)
    X = torch.zeros(usage_len, 1024, 39)
    framelen = torch.empty(usage_len)
    if mode == 'train':
        # y = torch.empty(max_len, dtype=torch.long)
        y = torch.zeros(usage_len, 1024, dtype=torch.long)

    idx = 0
    for i, fname in tqdm(enumerate(usage_list)):
        feat = load_feat(os.path.join(feat_dir, mode, f'{fname}.pt'))
        cur_len = len(feat)
        # feat = concat_feat(feat, concat_nframes)
        if mode == 'train':
          label = torch.LongTensor(label_dict[fname])

        # X[idx: idx + cur_len, :] = feat
        # X[idx: idx + cur_len, :, :] = feat
        X[i, :cur_len, :] = feat
        framelen[i] = cur_len
        if mode == 'train':
          # y[idx: idx + cur_len] = label
          #m = torch.nn.ConstantPad1d((0, 1024 - cur_len), -1)
          y[i, :cur_len] = label

        #idx += cur_len

    # X = X[:idx, :, :]
    # if mode == 'train':
    #   y = y[:idx]

    print(f'[INFO] {split} set')
    print(X.shape)
    if mode == 'train':
      print(y.shape)
      return X, y, framelen
    else:
      return X, framelen


# Dataset

In [30]:
import torch
from torch.utils.data import Dataset

class LibriDataset(Dataset):
    def __init__(self, X, y=None, framelen=None):
        self.data = X
        self.framelen = framelen
        if y is not None:
            self.label = torch.LongTensor(y)
        else:
            self.label = None

    def __getitem__(self, idx):
        if self.label is not None:
            return self.data[idx], self.label[idx], self.framelen[idx]
        else:
            return self.data[idx], self.framelen[idx]

    def __len__(self):
        return len(self.data)


# Model
Feel free to modify the structure of the model.

In [31]:
import torch.nn as nn

class BasicBlock(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(BasicBlock, self).__init__()

        # TODO: apply batch normalization and dropout for strong baseline.
        # Reference: https://pytorch.org/docs/stable/generated/torch.nn.BatchNorm1d.html (batch normalization)
        #       https://pytorch.org/docs/stable/generated/torch.nn.Dropout.html (dropout)
        self.block = nn.Sequential(
            nn.Linear(input_dim, output_dim),
            nn.BatchNorm1d(output_dim),
            nn.ReLU(),
            nn.Dropout(0.75),
        )
        # self.linear = nn.Linear(input_dim, 1024)
        # self.lrelu = nn.LeakyReLU()
        # self.bn = nn.BatchNorm1d(1024)
        # self.gru = nn.GRU(input_dim, hidden_size=512, bidirectional=True, batch_first=True)
        # self.lstm = nn.LSTM(1024, hidden_size=1024, bidirectional=True, batch_first=True)
        # self.dropout = nn.Dropout(0.5)
        # self.out = nn.Linear(1024, output_dim)

    def forward(self, x):
        x = self.block(x)
        # x = self.linear(x)
        # x = self.bn(x)
        # x, _ = self.gru(x)
        # x = self.dropout(x)
        # x = self.out(x)
        return x


class Classifier(nn.Module):
    def __init__(self, input_dim, output_dim=41, hidden_layers=1, hidden_dim=256):
        super(Classifier, self).__init__()

        self.fc = nn.Sequential(
            BasicBlock(input_dim, hidden_dim),
            *[BasicBlock(hidden_dim, hidden_dim) for _ in range(hidden_layers)],
            nn.Linear(hidden_dim, output_dim)
        )
        self.linear = nn.Linear(input_dim, 128)
        self.lrelu = nn.LeakyReLU()
        self.ln1 = nn.LayerNorm(input_dim)
        self.ln2 = nn.LayerNorm(hidden_dim * 2)
        self.gru = nn.GRU(input_dim, hidden_dim, hidden_layers, bidirectional=True, batch_first=True, dropout=0.5)
        self.lstm = nn.LSTM(input_dim, hidden_dim, hidden_layers, bidirectional=True, batch_first=True, dropout=0.5)
        self.softmax = nn.Softmax(dim = 1)
        self.out = nn.Linear(hidden_dim * 2, output_dim)

    def forward(self, x):
        #x = self.fc(x)
        #x = self.ln1(x)
        x, _ = self.gru(x)

        #x = self.ln2(x)

        #x = self.softmax(x)
        x = self.out(x)
        return x

# Hyper-parameters

In [32]:
# data prarameters
# TODO: change the value of "concat_nframes" for medium baseline
concat_nframes = 39   # the number of frames to concat with, n must be odd (total 2k+1 = n frames)
train_ratio = 0.95   # the ratio of data used for training, the rest will be used for validation

# training parameters
seed = 1213          # random seed
batch_size = 64        # batch size
num_epoch = 500         # the number of training epoch
learning_rate = 1e-3      # learning rate
model_path = './model.ckpt'  # the path where the checkpoint will be saved

# model parameters
# TODO: change the value of "hidden_layers" or "hidden_dim" for medium baseline
input_dim = 39# * concat_nframes  # the input dim of the model, you should not change the value
hidden_layers = 8          # the number of hidden layers
hidden_dim = 256           # the hidden dim

# Dataloader

In [33]:
from torch.utils.data import DataLoader
import gc

same_seeds(seed)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'DEVICE: {device}')

# preprocess data
train_X, train_y, train_len = preprocess_data(split='train', feat_dir='./libriphone/feat', phone_path='./libriphone', concat_nframes=concat_nframes, train_ratio=train_ratio, random_seed=seed)
val_X, val_y, val_len = preprocess_data(split='val', feat_dir='./libriphone/feat', phone_path='./libriphone', concat_nframes=concat_nframes, train_ratio=train_ratio, random_seed=seed)

# get dataset
train_set = LibriDataset(train_X, train_y, train_len)
val_set = LibriDataset(val_X, val_y, val_len)

# remove raw feature to save memory
del train_X, train_y, val_X, val_y, train_len, val_len
gc.collect()

# get dataloader
train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_set, batch_size=batch_size, shuffle=False)

DEVICE: cuda
[Dataset] - # phone classes: 41, number of utterances for train: 3257


3257it [00:03, 854.20it/s]


[INFO] train set
torch.Size([3257, 1024, 39])
torch.Size([3257, 1024])
[Dataset] - # phone classes: 41, number of utterances for val: 172


172it [00:00, 916.52it/s]

[INFO] val set
torch.Size([172, 1024, 39])
torch.Size([172, 1024])


# Training

In [34]:
# create model, define a loss function, and optimizer
model = Classifier(input_dim=input_dim, hidden_layers=hidden_layers, hidden_dim=hidden_dim).to(device)
criterion = nn.CrossEntropyLoss() 
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate, weight_decay=0.001)

scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=10, eta_min=1e-5)
best_acc = 0.0
for epoch in range(num_epoch):
    train_acc = 0.0
    train_loss = 0.0
    val_acc = 0.0
    val_loss = 0.0
    train_totalframelen = 0
    val_totalframelen = 0
    
    # training
    model.train() # set the model to training mode
    for i, batch in enumerate(tqdm(train_loader)):
        features, labels, framelen = batch
        features = features.to(device)
        labels = labels.to(device)
        
        optimizer.zero_grad() 
        outputs = model(features)
        outputs = torch.transpose(outputs, 1, 2)
        # print(outputs.shape)
        # print(labels.shape)

        # for j in range(framelen.shape[0]):
        #     loss = criterion(outputs[j, :int(framelen[j]), :], labels[j, :int(framelen[j])])
        #     train_loss += loss.item()

        loss = criterion(outputs, labels)
        loss.backward() 
        optimizer.step() 
        
        _, train_pred = torch.max(outputs, 1) # get the index of the class with the highest probability

        for j in range(framelen.shape[0]):
            train_acc += (train_pred[j, :int(framelen[j])].detach() == labels[j, :int(framelen[j])].detach()).sum().item()
        train_totalframelen += framelen.sum().item()
        #train_acc += (train_pred.detach() == labels.detach()).sum().item()
        train_loss += loss.item()

    #train_loss /= 64.0
    train_acc /= (train_totalframelen / 3257)
    
    # validation
    model.eval() # set the model to evaluation mode
    with torch.no_grad():
        for i, batch in enumerate(tqdm(val_loader)):
            features, labels, framelen = batch
            features = features.to(device)
            labels = labels.to(device)
            outputs = model(features)
            outputs = torch.transpose(outputs, 1, 2)
            
            # for j in range(framelen.shape[0]):
            #     loss = criterion(outputs[j, :int(framelen[j]), :], labels[j, :int(framelen[j])])
            #     val_loss += loss.item()

            loss = criterion(outputs, labels) 
            
            _, val_pred = torch.max(outputs, 1) 

            for j in range(framelen.shape[0]):
                val_acc += (val_pred[j, :int(framelen[j])].cpu() == labels[j, :int(framelen[j])].cpu()).sum().item()
            val_totalframelen += framelen.sum().item()
            #val_acc += (val_pred.cpu() == labels.cpu()).sum().item() # get the index of the class with the highest probability
            val_loss += loss.item()
    
    #val_loss /= 64.0
    val_acc /= (val_totalframelen / 172)

    scheduler.step()
    print(f'[{epoch+1:03d}/{num_epoch:03d}] Train Acc: {train_acc/len(train_set):3.5f} Loss: {train_loss/len(train_loader):3.5f} | Val Acc: {val_acc/len(val_set):3.5f} loss: {val_loss/len(val_loader):3.5f}')

    # if the model improves, save a checkpoint at this epoch
    if val_acc > best_acc:
        best_acc = val_acc
        torch.save(model.state_dict(), model_path)
        print(f'saving model with acc {best_acc/len(val_set):.5f}')


100%|██████████| 3/3 [00:00<00:00,  4.20it/s]


[001/500] Train Acc: 0.35005 Loss: 1.50679 | Val Acc: 0.48674 loss: 1.05583
saving model with acc 0.48674


100%|██████████| 3/3 [00:00<00:00,  4.04it/s]


[002/500] Train Acc: 0.54792 Loss: 0.89712 | Val Acc: 0.59956 loss: 0.79956
saving model with acc 0.59956


100%|██████████| 3/3 [00:00<00:00,  4.56it/s]


...

[213/500] Train Acc: 0.95672 Loss: 0.06685 | Val Acc: 0.83333 loss: 0.48203


100%|██████████| 3/3 [00:00<00:00,  3.91it/s]


[214/500] Train Acc: 0.95682 Loss: 0.06657 | Val Acc: 0.83294 loss: 0.48723


 86%|████████▋ | 44/51 [00:34<00:05,  1.28it/s]


KeyboardInterrupt: 

In [35]:
del train_set, val_set
del train_loader, val_loader
gc.collect()

517

# Testing
Create a testing dataset, and load model from the saved checkpoint.

In [36]:
# load data
test_X, test_len = preprocess_data(split='test', feat_dir='./libriphone/feat', phone_path='./libriphone', concat_nframes=concat_nframes)
test_set = LibriDataset(test_X, None, test_len)
test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False)

[Dataset] - # phone classes: 41, number of utterances for test: 857


857it [00:00, 1464.94it/s]

[INFO] test set
torch.Size([857, 1024, 39])


In [37]:
# load model
model = Classifier(input_dim=input_dim, hidden_layers=hidden_layers, hidden_dim=hidden_dim).to(device)
model.load_state_dict(torch.load(model_path))

<All keys matched successfully>

Make prediction.

In [38]:
pred = np.array([], dtype=np.int32)

model.eval()
with torch.no_grad():
    for i, batch in enumerate(tqdm(test_loader)):
        features, framelen = batch
        features = features.to(device)

        outputs = model(features)
        outputs = torch.transpose(outputs, 1, 2)

        _, test_pred = torch.max(outputs, 1) # get the index of the class with the highest probability
        
        for j in range(framelen.shape[0]):
            pred = np.concatenate((pred, test_pred[j][:int(framelen[j])].cpu().numpy()), axis=0)

        #pred = np.concatenate((pred, test_pred.cpu().numpy()), axis=0)


100%|██████████| 14/14 [00:03<00:00,  3.73it/s]


Write prediction to a CSV file.

After finish running this block, download the file `prediction.csv` from the files section on the left-hand side and submit it to Kaggle.

In [39]:
with open('prediction.csv', 'w') as f:
    f.write('Id,Class\n')
    for i, y in enumerate(pred):
        f.write('{},{}\n'.format(i, y))